In [1]:
import os
import json

from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate

from agent.utils.loader import load_prompt, load_processed_data
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts.chat import BaseMessage

_ = load_dotenv(find_dotenv())

In [9]:
dataset_name = 'trivia_qa'
mode = "direct"
model = "gpt-4o-mini"
num_samples = 1000
top_p = 0.95
temperature = 0
seed = 42
batch_size = 100
if dataset_name == "toxicity":
	temperature = 0.9
	n = 25
else:
	temperature = 0
	n = 1
processed_data_path = f"../../data/processed_data/{dataset_name}.jsonl"
save_results_path = f"../../output/inference/{model}/{dataset_name}/{mode}/num_samples_{num_samples}_top_p_{top_p}_temperature_{temperature}.jsonl"
prompt = load_prompt(dataset_name=dataset_name, mode=mode)
dataset = load_processed_data(dataset_name=dataset_name, file_path=processed_data_path)
llm = ChatOpenAI(model=model, top_p=top_p, n=n, temperature=temperature, base_url="https://api.chsdw.top/v1", seed=seed)

prompt.pretty_print()
print(dataset)

C:\Users\Administrator\miniconda3\envs\self-improve\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

================================ System Message ================================

Remember your answer should follow previous pattern.

================================ Human Message =================================

Q: Mendelssohn's 'Wedding March' was. originally written as incidental music for which Shakespeare play in 1842?

================================== AI Message ==================================

A: A Midsummer Night's Dream

================================ Human Message =================================

Q: """Christ in the House of his Parents"" is one of the best known paintings of which artist?"

================================== AI Message ==================================

A: John Millais

================================ Human Message =================================

Q: Who designed the National Theatre building on the South Bank in London ?

================================== AI Message ==================================

A: Sir Denys Lasdun

================

In [10]:
from tqdm.asyncio import tqdm, tqdm_asyncio
import nest_asyncio

nest_asyncio.apply()

async def inference(item: dict) -> str:
	try:
		response: LLMResult = await llm.agenerate(messages=[prompt.invoke(input=item)])
		if dataset_name in ["hotpot_qa", "trivia_qa", "ambig_qa"]:
			response_content = response.generations[0][0].message.content
			result = {**item, "prediction": response_content.split("A: ")[-1]}
		elif dataset_name in ["gsm9k", "tabmwp", "svamp"]:
			response_content = response.generations[0][0].message.content
			result = {**item, "prediction": response_content.split("Answer: ")[-1]}
		elif dataset_name == "toxicity":
			responses_content = [choice.message.content for choice in response.generations[0]]
			result = {**item, "prediction": responses_content}
	except Exception as e:
		print(e)
		result = {**item, "prediction": "ERROR"}
	return result

async def direct_inference() -> None:
	results = []
	if os.path.exists(save_results_path):
	    with open(save_results_path, 'r') as file:
	        for line in file:
	            results.append(json.loads(line))
	else:
		folder_path = os.path.dirname(save_results_path)
		os.makedirs(folder_path, exist_ok=True)
	
	for idx in tqdm(range(len(results), num_samples, batch_size)):
		batch = dataset.select(range(idx, min(idx+batch_size, num_samples)))
		results.extend(await tqdm_asyncio.gather(*(inference(item) for item in batch)))
		with open(save_results_path, 'w') as file:
			for result in results:
				file.write(json.dumps(result) + "\n")
				
await direct_inference()

100%|██████████| 10/10 [00:55<00:00,  5.54s/it]
